In [ ]:
import os
import shutil
import numpy as np
import ffmpeg
import PIL
from PIL import Image
import spectrogramFuncs as spec
import midiCreationFuncs as midiMagic
import makeDataset as md
import pretty_midi as pm

In [ ]:
## SINGLE Instrument 120 BPM Ascension MIDI file creation
## Can be used to synthesise audio using soundfounts in programs such as MuseScore

# Path to save folder (omit final /)
folder = 'CreationTest/Single'  
file_name = '4Asc'
note_length = 0.5 # Crotchet
instrument_MIDI_program_number = 27 # Clean electric guitar
# MIDI note range: 0 = C0, 127 = G9
lowest_note = 40
highest_note = 88

midiMagic.create_instrument_ascension(file_name, folder, noteLength = note_length, instrument_prog_num = instrument_MIDI_program_number, lowestNote = lowest_note, highestNote = highest_note) 

In [ ]:
## DUAL Instrument 120 BPM Ascension MIDI file creation
## Can be used to synthesise audio using soundfounts in programs such as MuseScore

# Path to save folder (omit final /)
folder = 'CreationTest/Double'  
file_name = '4AscDual'
note_length = 0.5 # Crotchet
first_instrument_MIDI_program_number = 27 # Clean electric guitar
second__instrument_MIDI_program_number = 56 # Trumpet
# MIDI note range: 0 = C0, 127 = G9
first_note_range = [40,88]
second_note_range = [54,86]

midiMagic.create_two_instrument_ascension(file_name, folder, noteLength = note_length, firstProg = first_instrument_MIDI_program_number, secondProg = second__instrument_MIDI_program_number, firstRange = first_note_range, secondRange = second_note_range) 

In [ ]:
## Proceed once .wav files have been synthesised from the MIDI files
# If you are not using guitar and trumpet as your instruments, see the "midiCreationFuncs.py" file for the labelling functions and adjust them accordingly.

# Get appropriate files here, omit final /
folder_containing_files = 'CreationTest/Single'
# Best way to organise folders is to have one midi file per folder with a collection of .wav files
midi_path, wav_paths = midiMagic.process_folder_with_midi_and_wavs(folder_containing_files)

In [ ]:
## TWO Instrument labelling (make sure that order of instruments in midi_file is consistent) -- see lower down for ONE instrument labelling
# If you are not using guitar and trumpet as your instruments, see the "midiCreationFuncs.py" file for the labelling functions and adjust them accordingly.
labels = [[],[]]
datas = np.ndarray(shape=(0,512,128,3), dtype='float32', order='C')
fft_window = 2048 # The spectrogram frame size will be twice this value (but this value will appear next to the spectrogram folder)
hop_length = 512
# Note length of ascension in MIDI file
note_length = 0.5
# Labelling
repeat = 0 # increase if you'd like to have multiple copies of the same data
while repeat >= 0:
    for wav in wav_paths:
        guit, trump = midiMagic.create_inst_label(midi_path, note_length, (frame_size)/32768) 

        spec.create_spectrogram_set_from_wav_file(folder_containing_files + '/' + wav, hopLength = hop_length, fwindow = int(fft_window), reset=False)  

        data = md.spectrograms_to_arrays(folder_containing_files + '/' + wav + 'Spectrograms' + 'F_' + str(frame_size) + 'H_' + str(hop_length))

        # Add labels for nothing, let = label 129
        nothingLength = len(data)-len(guit)
        while nothingLength > 0:
            guit.append([0]*128 + [1])
            trump.append([0]*128 + [1])
            nothingLength -= 1
        #data = data[:len(label_0)]
        
        labels[0].extend(guit)
        labels[1].extend(trump)
        datas = np.concatenate((datas, data), axis = 0)
    repeat -= 1

print("Complete")

In [ ]:
## Create TWO instrument dataset
folder_path = '' # omit final /
dataset_name = ''

labels = np.array(labels)
md.create_two_label_dataset(folder_path + '/', dataset_name, datas, labels)

In [ ]:
## FOR TROUBLESHOOTING (TWO INST)
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
## FOR TROUBLESHOOTING (TWO INST)
for i,j in zip(labels[0], labels[1]):
    print(np.argmax(i), np.argmax(j))
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
## ONE instrument labelling
# If you are not using guitar and trumpet as your instruments, see the "midiCreationFuncs.py" file for the labelling functions and adjust them accordingly.
label_ins = 'Guitar'
labels = []
datas = np.ndarray(shape=(0,512,128,3), dtype='float32', order='C')
fft_window = 2048 # The spectrogram frame size will be twice this value
hop_length = 512
# Note length used in the midi-ascension file
note_length = 0.5
# Labelling
repeat = 0 # increase if you'd like to have multiple copies of the same data
while repeat >= 0:
    for wav in wav_paths:
        inst = midiMagic.create_sole_inst_label(midi_path, note_length, (frame_size)/32768, label_instrument=label_ins) 
        
        spec.create_spectrogram_set_from_wav_file(folder_containing_files + '/' + wav, hopLength = hop_length, fwindow = int(fft_window), reset=False)     

        data = md.spectrograms_to_arrays(folder_containing_files + '/' + wav + 'Spectrograms' + 'F_' + str(frame_size) + 'H_' + str(hop_length))

        # Add labels for nothing, let = label 129
        nothingLength = len(data)-len(inst)
        while nothingLength > 0:
            inst.append([0]*128 + [1])
            nothingLength -= 1
        #data = data[:len(label_0)]
        
        labels.extend(inst)
        datas = np.concatenate((datas, data), axis = 0)
    repeat -= 1

print("Complete")

In [ ]:
## Create ONE instrument Dataset
folder_path = 'CreationTest' # omit final /
dataset_name = 'TestSingleOne'

labels = np.array(labels)
md.create_one_label_dataset(folder_path + '/' + dataset_name, datas, labels) 

In [ ]:
## FOR TROUBLESHOOTING (ONE INST)
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
## FOR TROUBLESHOOTING (ONE INST)
for i in range(len(labels)):
    print(np.argmax(labels[i]),)
print(np.shape(labels))
print(np.shape(datas))